In [1]:
import flowenv
import matplotlib
import random

from collections import deque, namedtuple
from matplotlib import animation
from torch.distributions import Categorical

import gymnasium as gym
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.utils as utils
import torch.optim as optim
import pandas as pd

is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

In [13]:
TRAIN_DATA_PATH = "DNP3_Intrusion_Detection_Dataset_Final/Training_Testing_Balanced_CSV_Files/CICFlowMeter/CICFlowMeter_Training_Balanced.csv"
TEST_DATA_PATH = "DNP3_Intrusion_Detection_Dataset_Final/Training_Testing_Balanced_CSV_Files/CICFlowMeter/CICFlowMeter_Testing_Balanced.csv"

# CSVファイルの読み込み
raw_data_train = pd.read_csv(TRAIN_DATA_PATH).replace([np.inf, -np.inf], np.nan).dropna(how="all").dropna(how="all", axis=1)
raw_data_test = pd.read_csv(TEST_DATA_PATH).dropna(how="all").replace([np.inf, -np.inf], np.nan).dropna(how="all", axis=1)

raw_data_train["Binary Label"] = raw_data_train["Label"] == "NORMAL"
raw_data_test["Binary Label"] = raw_data_test["Label"] == "NORMAL"

train_env = gym.make("flowenv/FlowTrain-v0", data=raw_data_train)
# test_env = gym.make("flowenv/FlowTest-v0", data=raw_data_test)

In [3]:
def min_max_p(p):
    min_p = p.min()
    max_p = p.max()
    return (p - min_p) / (max_p - min_p)

In [4]:
# raw_data_train["Flow Duration"] = min_max_p(raw_data_train["Flow Duration"])

In [14]:
from flowenv.const import Const

labels = Const().normalization_features

# print(labels)
except_labels = [
    "ECE Flag Cnt",
    "Bwd Byts/b Avg",
    "FIN Flag Cnt",
    "RST Flag Cnt",
]

for label in Const().normalization_features:
    # print(label)
    if label in except_labels:
        continue
    raw_data_train[label] = min_max_p(raw_data_train[label])

raw_data_train

,Unnamed: 0.1,Unnamed: 0,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,...,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label,Binary Label
0,38416,108,192.168.1.5-192.168.1.12-20003-57624-6,192.168.1.12,57624,192.168.1.5,20003,6,19/05/2020 03:16:30 AM,0.994675,...,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,INIT_DATA,False
1,2081,602,192.168.1.1-192.168.1.9-49367-20001-6,192.168.1.1,49367,192.168.1.9,20001,6,15/05/2020 02:00:05 AM,0.983834,...,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,DISABLE_UNSOLICITED,False
2,19562,906,192.168.1.7-192.168.1.12-20003-59969-6,192.168.1.12,59969,192.168.1.7,20003,6,16/05/2020 02:56:06 AM,0.983841,...,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,WARM_RESTART,False
3,40475,65,192.168.1.2-192.168.1.3-54139-20000-6,192.168.1.2,54139,192.168.1.3,20000,6,18/05/2020 03:13:14 PM,0.733082,...,0.000000,0.000000,0.000000,0.000000,7.499957e-01,7.071043e-01,9.999943e-01,1.069430e-08,MITM_DOS,False
4,40693,93,8.0.6.4-8.6.0.1-0-0-0,8.6.0.1,0,8.0.6.4,0,0,18/05/2020 03:55:08 PM,0.999717,...,0.011993,0.000000,0.011993,0.011993,3.854689e-09,0.000000e+00,3.854689e-09,3.854689e-09,ARP_POISONING,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5121,40546,136,192.168.1.2-192.168.1.3-54139-20000-6,192.168.1.2,54139,192.168.1.3,20000,6,18/05/2020 04:37:03 PM,0.204831,...,0.000000,0.000000,0.000000,0.000000,1.546075e-08,0.000000e+00,1.546075e-08,1.546075e-08,MITM_DOS,False
5122,40811,33,8.0.6.4-8.6.0.1-0-0-0,8.6.0.1,0,8.0.6.4,0,0,18/05/2020 02:36:40 PM,0.995936,...,0.149222,0.393163,0.330002,0.037838,4.104721e-09,8.149570e-10,4.703363e-09,3.553812e-09,ARP_POISONING,False
5123,40758,158,192.168.1.2-192.168.1.4-48948-20000-6,192.168.1.2,48948,192.168.1.4,20000,6,18/05/2020 05:22:43 PM,0.540543,...,0.000000,0.000000,0.000000,0.000000,3.899834e-08,0.000000e+00,3.899834e-08,3.899834e-08,MITM_DOS,False
5124,40481,71,192.168.1.2-192.168.1.3-54139-20000-6,192.168.1.2,54139,192.168.1.3,20000,6,18/05/2020 03:20:51 PM,0.473125,...,0.000000,0.000000,0.000000,0.000000,3.571175e-08,0.000000e+00,3.571175e-08,3.571175e-08,MITM_DOS,False
